<p style="text-align: center;">
    Министерство науки и высшего образования Российской Федерации<br>
    Федеральное государственное автономное образовательное учреждение высшего образования «Национальный исследовательский Нижегородский государственный университет им. Н.И. Лобачевского»<br>
    <br>
    Институт Информационных технологий, математики и механики<br>
    <br><br><br>
    Дипломная работа<br>
    <h1 style="text-align: center;">Исследование сигнала ЭКГ с целью выявления сердечно-сосудистых заболеваний посредством нейросетей</h1>
</p>
<br><br><br><br><br>
<p style = "text-align: left; margin-left: 80%;">
    Выполнил:<br>
    студент гр. 381803-1<br>
    <p style = "text-align: left; margin-left: 90%;">
        Мешалкин Н.А.
    </p>
</p>
<br><br><br>
<p style = "text-align: left; margin-left: 80%;">
    Проверил:<br>
    директор ИИТММ, заведующий кафедрой АГДМ<br>
    <p style = "text-align: left; margin-left: 90%;">
        Золотых Ю.Н.
    </p>
</p>
<br><br><br><br>
<p style="text-align: center">
    Нижний Новгород<br>
    2021
</p>

# Введение

**Описание работы:**
Работа осуществляется с недавно открытой (2019г.) исследовательской базой данных под эгидой Chapman University и Shaoxing People's Hospital для сигналов электрокардиограмм в 12 отведениях. База данных направлена на исследования аритмии и других сердечно-сосудистых заболеваний. Некоторые виды аритмий, такие как мерцательная аритмия, оказывают выраженное негативное влияние на здоровье населения и качество жизни в целом. Так как в современном мире работа осуществляется с большим объемом данных, не всегда исследование "вручную" врачами-кардиологами возможно. Современные инструменты машинного обучения и статистики могут помочь врачам в исследовании. В связи с этим, работа направлена на исследование базы данных, содержащую ЭКГ в 12 отведениях 10 646 пациентов с частотой дискретизации 500 Гц, которая включает 11 распространенных ритмов и 67 дополнительных сердечно-сосудистых заболеваний, все помечены профессиональными экспертами. Набор данных состоит из 10-секундных 12-мерных наборов ЭКГ и соответсвующих меток для каждого объекта.

**Цель работы:**
Необходимо "обработать" голый сигнал, измеренный в 12 отведениях с частотой дискретизации 500 Гц для 10646 пациентов. Под словом "обработать" стоит иметь ввиду применение сегментируюшей нейросети к нашему сигналу. После получения меток для каждого образца, измерить интересующие нас характеристики (метрики). Далее следует проверить метрики, которые были получены уже на размеченных данных из базы данных. Сравнить результаты.

Данная работа состоит из двух частей:
1) Работа с размеченными данными, проверка характеристик.
2) Работа с голым сигналом, применение к голому сигналу сегментирующей нейросети ННГУ,получение размеченных данных, проверка результатов.

## Часть 1:

Рассмотрим размеченную базу данных, содержащую ЭКГ в 12 отведениях 10 646 пациентов с частотой дискретизации 500 Гц, которая включает 11 общих ритмов и 67 дополнительных сердечно-сосудистых заболеваний.

Признаки датасета, их свойства и пояснения приведены в таблице №1.

                                                            Таблица №1.

*Attributes* | *Type* | *Value Range* | *Description* |
--- | --- | --- | --- |
FileName | String |  | ECG data file name |
Rhythm | String |  | Rhythm Label |
Beat | String |  | Other conditions Label |
PatientAge | Numeric | 0-999 | Age |
Gender | String | MALE/FEMALE | Gender |
VentricularRate | Numeric | 0-999 | Ventricular rate in BPM |
AtrialRate | Numeric | 0-999 | Atrial rate in BPM |
QRSDuration | Numeric | 0-999 | QRS duration in BPM |
QTInterval | Numeric | 0-999 | QT interval in msec |
QTTCorrected | Numeric | 0-999 | Corrected QT interval in msec |
RAxis | Numeric | -179~180 | R axis |
TAxis | Numeric | -179~181 | T axis |
QRSCount | Numeric | 0-254 | QRS count |
QOnset | Numeric | 16 Bit Unsigned | Q onset |
QOffset | Numeric | 17 Bit Unsigned | Q offset |
TOffset | Numeric | 18 Bit Unsigned | T offset |

Именно такие признаки были получены в ходе извлечения признаков у разработчиков базы данных. К основным признакам добавили возраст и пол из-за их важности почти во всех анализах медицинских данных. Характеристики, извлеченные II-го отведения, включают частоту желудочков в ударах в минуту (VentricularRate), предсердную частоту в ударах в минуту (AtrialRate), продолжительность QRS в миллисекундах (QRSDuration), интервал QT в миллисекундах (QTInterval), ось R (RAxis), ось T (TAxis), количество QRS (QRSCount), начало Q (QOnset), смещение Q (QOffset), среднее значение интервала RR, дисперсия интервала RR, количество интервалов RR.

Таким образом, наш датасет состоит из 10646 объектов/сэмплов (пациенты), у которых признаками являются признаки, полученные в результате исследования голого сигнала ЭКГ (признаки приведены в таблице №1). Нам необходимо будет предсказывать посредством алгоритмов машинного обучения признак Rhytnm, который отображает нарушение сердечного ритма у данного пациента.

Прочитаем данные:

In [ ]:
import sklearn
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("data/Diagnostics.csv",delimiter=';',index_col='FileName')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
data


In [ ]:
data.loc["MUSE_20180114_075129_97000"]

Нас интересует столбец "Rhythm". Рассмотрим его значения подробнее.

In [ ]:
data["Rhythm"].value_counts()

Видим, что присутствует дисбаланс классов. Чтобы углубиться в проблему, рассмтотрим наши метки и разберемся, что они из себя представляют.
Классы нашего датасета представлены в таблице №2.


                                                           Таблица №2.

*Acronym Name* | *Full Name* | *Frequency,n(%)* |
--- | --- | --- |
SB | Sinus Bradycardia | 3.88(36.53) |
SR | Sinus Rhythm | 1.82(17.15) |
Beat | Atrial Fibrillation | 1.7(16.72) |
PatientAge | Sinus Tachycardia | 1.5(14.73) |
Gender | Atrial Flutter | 445 (4.18) |
VentricularRate | Sinus Irregularity | 399 (3.75) |
AtrialRate | Supraventricular Tachycardia | 587 (5.51) |
QRSDuration | Atrial Tachycardia | 121 (1.14) |
QTInterval | Atrioventicular Node Reentrant Tachycardia | 16 (0.15) |
QTTCorrected | Atrioventicular Reentrant Tachycardia | 8 (0.07) |
RAxis | Sinus Atrium to Atrial Wandering Rhythm | 7 (0.07) |
All | All | 10.646 (100) |


Поскольку некоторые ритмы имеют крайне мало образцов, по предложению кардиологов следует объединить несколько редких случаев в типы аритмий верхнего уровня.
Таким образом, 11 ритмов были объединены в 4 группы (SB, AFIB, GSVT, SR), представленные в таблице 3.

**SB включает:**
1) Синусовую брадикардию (sinus bradycardia (SB))

**AFIB состоит:**
1) Фибрилляции предсердий (atrial fibrillation (AFIB)),
2) Трепетания предсердий (and atrial flutter (AF))

**GSVT содержит:**
1) Наджелудочковая тахикардия (supraventricular tachycardia (SVT)),
2) Предсердная тахикардия( atrial tachycardia (AT)),
3) Атриовентрикулярная возвратная тахикардия в узле(atrioventricular node reentrant tachycardia (AVNRT)),
4) Возвратная атриовентрикулярная тахикардия(atrioventricular reentrant tachycardia (AVRT)),
5) Синусовое предсердие с блуждающим ритмом предсердий(sinus atrium to atrial wandering rhythm (SAAWR)),
6) Синусовая тахикардия (sinus Tachycardia (ST))

**SR включает:**
1) Синусовый ритм (sinus rhythm (SR))
2) Нерегулярность синусового узла (sinus irregularity (SI))

                                                  Таблица №3. Итоговые классы.

*Merged from* | *Merged to* | *Total* |
--- | --- | --- |
AFIB,AF | AFIB | 3,889 |
SVT,AT,SAAWR,ST,AVNRT,AVRT | GSVT | 2,307 |
SB | SB | 2,225 |
SR,SI | SR | 2,225 |
All | All | 10,646 |



Объединим классы:

In [ ]:
for i in range(0,len(data["Rhythm"])):
    if data['Rhythm'][i]=='AFIB' or data['Rhythm'][i]=='AF':
        data['Rhythm'][i]='AFIB'
    if data['Rhythm'][i]=='SVT' or data['Rhythm'][i]=='AT' or data['Rhythm'][i]=='AVNRT' or data['Rhythm'][i]=='AVRT'  or data['Rhythm'][i]=='ST' or data['Rhythm'][i]=='SAAWR':
        data['Rhythm'][i]='GSVT'
    if data['Rhythm'][i]=='SR' or data['Rhythm'][i]=='SA':
        data['Rhythm'][i]='SR'

In [ ]:
data['Rhythm'].value_counts()

### Работа с данными:

##### Проверка коррелирующих признаков

In [ ]:
corr_mat = data.corr()
corr_mat

In [ ]:
sns.heatmap(corr_mat, square=True, cmap='coolwarm')
pass

Из матрицы корреляции четко видно что, есть некоторое количество коррелирующих между собой признаков. Рассмотрим их подробнее.

In [ ]:
corr_mat.where(np.triu(corr_mat > 0.75, k=1)).stack().sort_values(ascending=False)

Удалим коррелирующие признаки из нашего датасета.

In [ ]:
data=data.drop(['TOffset'], axis = 1)
data=data.drop(['QRSCount'], axis = 1)

corr_mat = data.corr()
corr_mat.where(np.triu(corr_mat > 0.75, k=1)).stack().sort_values(ascending=False)


#####  Приведение данных к одному типу

In [ ]:
data_notNull = data.drop(['Beat'], axis=1) #Удаление столбца Beat, из-за большого числа пропущенных значений
data_notNull

Воспользуемся функцией Label Encoder. Данная функция маркирует признаки с нечисловыми типами данных и приводит их к числовым.

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(data['Gender'])
data_notNull['Gender']=le.transform(data_notNull['Gender'])
le.fit(data_notNull['Rhythm'])
data_notNull['Rhythm']=le.transform(data_notNull['Rhythm'])
data_notNull=pd.get_dummies(data_notNull, columns=["Gender"])

In [ ]:
data_notNull

##### Проверка выбросов


Проверим наиболее для нас важные признаки и удалим те объекты из нашего датасета, которые выходят за интервал квантиля (0.005;0.995)

In [ ]:
plt.scatter(data_notNull['Rhythm'],data_notNull['PatientAge'])
pass

In [ ]:
rows_to_drop = data_notNull[(data_notNull['PatientAge'] < data_notNull['PatientAge'].quantile(0.005)) |
                            (data_notNull['PatientAge'] > data_notNull['PatientAge'].quantile(0.995))].index
data_notNull = data_notNull.drop(rows_to_drop)
data_notNull.shape

In [ ]:
plt.scatter(data_notNull['Rhythm'],data_notNull['VentricularRate'])
pass

In [ ]:
rows_to_drop = data_notNull[(data_notNull['VentricularRate'] < data_notNull['VentricularRate'].quantile(0.005)) |
                            (data_notNull['VentricularRate'] > data_notNull['VentricularRate'].quantile(0.995))].index
data_notNull = data_notNull.drop(rows_to_drop)
data_notNull.shape

In [ ]:
plt.scatter(data_notNull['Rhythm'],data_notNull['AtrialRate'])
pass

In [ ]:
rows_to_drop = data_notNull[(data_notNull['AtrialRate'] < data_notNull['AtrialRate'].quantile(0.005)) |
                            (data_notNull['AtrialRate'] > data_notNull['AtrialRate'].quantile(0.995))].index
data_notNull = data_notNull.drop(rows_to_drop)
data_notNull.shape

In [ ]:
plt.scatter(data_notNull['Rhythm'],data_notNull['RAxis'])
pass

In [ ]:
rows_to_drop = data_notNull[(data_notNull['RAxis'] < data_notNull['RAxis'].quantile(0.005)) |
                            (data_notNull['RAxis'] > data_notNull['RAxis'].quantile(0.995))].index
data_notNull = data_notNull.drop(rows_to_drop)
data_notNull.shape

In [ ]:
plt.scatter(data_notNull['Rhythm'],data_notNull['QTInterval'])
pass

In [ ]:
rows_to_drop = data_notNull[(data_notNull['QTInterval'] < data_notNull['QTInterval'].quantile(0.005)) |
                            (data_notNull['QTInterval'] > data_notNull['QTInterval'].quantile(0.995))].index
data_notNull = data_notNull.drop(rows_to_drop)
data_notNull.shape

In [ ]:
data_notNull.pop('TAxis')
data_notNull.pop('RAxis')
print("")

### Разделение данных на тренировочную и тестовую выборки:

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
import imblearn

Выделем наш целевой столбец.

In [ ]:
y=data_notNull['Rhythm']
data_notNull.pop('Rhythm')
X=data_notNull

In [ ]:
y.value_counts()

Посмотрим на наши 4 итоговых класса. Видим, что есть небольшая несбалансированность классов (Но для частоты эксперимента не будем с ней бороться).

Проведем нормализацию данных.

In [ ]:
from sklearn.preprocessing import scale
X_scaled=scale(X)
X_scaled

Разделим данные на тестовую (25%) и тренировочную выборки (75%).

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25,stratify=y, random_state=42)
#мое

In [ ]:
X_train.shape

In [ ]:
y_train.value_counts()

#### Тренировка и оценка модели.

В качестве метрик рассмотрим f1score и confusion matrix. В данных условиях они являются более репрезетативными.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

#### SVC

In [ ]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train, y_train)

In [ ]:
y_pred=clf.predict(X_test)

print("recall test: ",recall_score(y_test.values, y_pred,average='macro'))
print("presion test: ",precision_score(y_test.values,y_pred ,average='macro'))
print("f1_score test: ",f1_score(y_test.values, y_pred,average='macro'))

In [ ]:
confusion_matrix(y_test.values, y_pred)
conf_mx = confusion_matrix(y_test.values, y_pred) #посылаем предсказанные значения и истинные
conf_mx

In [ ]:
plt.matshow(conf_mx, cmap=plt.cm.Blues)
plt.show()

#### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit
clf = KNeighborsClassifier()
cv = ShuffleSplit(n_splits=3, random_state=42)

clf.fit(X_train, y_train)

In [ ]:
y_pred=clf.predict(X_test)

print("recall test: ",recall_score(y_test.values, y_pred,average='macro'))
print("presion test: ",precision_score(y_test.values,y_pred ,average='macro'))
print("f1_score test: ",f1_score(y_test.values, y_pred,average='macro'))

In [ ]:
confusion_matrix(y_test.values, y_pred)
conf_mx = confusion_matrix(y_test.values, y_pred)
conf_mx

In [ ]:
plt.matshow(conf_mx, cmap=plt.cm.Blues)
plt.show()

#### RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

In [ ]:
y_pred=clf.predict(X_test)

print("recall test: ",recall_score(y_test.values, y_pred,average='macro'))
print("presion test: ",precision_score(y_test.values,y_pred ,average='macro'))
print("f1_score test: ",f1_score(y_test.values, y_pred,average='macro'))

In [ ]:
confusion_matrix(y_test.values, y_pred)
conf_mx = confusion_matrix(y_test.values, y_pred)
conf_mx

In [ ]:
plt.matshow(conf_mx, cmap=plt.cm.Blues)
plt.show()

#### GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)

In [ ]:
y_pred=clf.predict(X_test)

print("recall test: ",recall_score(y_test.values, y_pred,average='macro'))
print("presion test: ",precision_score(y_test.values,y_pred ,average='macro'))
print("f1_score test: ",f1_score(y_test.values, y_pred,average='macro'))


In [ ]:
confusion_matrix(y_test.values, y_pred)
conf_mx = confusion_matrix(y_test.values, y_pred)
conf_mx

In [ ]:
plt.matshow(conf_mx, cmap=plt.cm.Blues)
plt.show()

#### Neural network

In [ ]:
#Подключаем библиотеки
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
X

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(10,)),
    keras.layers.Dense(300, activation=tf.nn.tanh),
    keras.layers.Dense(200, activation=tf.nn.tanh),
    keras.layers.Dense(80, activation=tf.nn.tanh),
    keras.layers.Dense(4, activation=tf.nn.softmax)
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.fit(X_train, y_train, epochs=20)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

Лучшим алгоритмом является нейронная сеть. Она показала лучшую точность ~92%.
Таким образом мы провели исследование размеченных данных, которые нам были предоставлены базой данных Chapman University и Shaoxing People's Hospital. Теперь нам следует собственноручно провести исследование над голым сигналом и получить свои размеченные данные.

## Часть 2:

Рассмотрим сигналы электрокардиограммы в 12 отведениях (5000 образцов). Напомним, что для каждого испытуемого доступен сигнал длиной в 10 секунд. Общее количество испытуемых: 10646 человек.

Рассмотрим, случайно взятого пациента.

In [ ]:
data = pd.read_csv("ECGData/MUSE_20180111_155115_19000.csv",delimiter=',')
data

Напомню, что мы будем использовать нейронную сеть для сегментации сигналов на p, qrs и t сегменты. Результат сохраняется в виде формата json там же, где лежат сами сигналы.

In [ ]:
# import json
# import matplotlib.pyplot as plt
# import numpy as np
# import torch
# import pandas as pd
# import wfdb

# sample_rate = 500
# path_to_model = "Segmentation/"
# path_to_data = "data/"

# class Delineation:
#     def __init__(self):
#         self._model = torch.jit.load(path_to_model + "model.pt")
#         self._max_dist = int(0.03 * sample_rate)
#         self._border = int(0.8 * sample_rate)

#     def _remove_small(self, signal):
#         last_zero = 0
#         for i in range(len(signal)):
#             if signal[i] == 0:
#                 if i - last_zero < self._max_dist:
#                     signal[last_zero:i] = 0
#                 last_zero = i

#     def _merge_small(self, signal):
#         lasts = np.full(signal.max() + 1, -(self._max_dist+1))
#         for i in range(len(signal)):
#             m = signal[i]
#             if i - lasts[m] < self._max_dist and m > 0:
#                 signal[lasts[m]:i] = m
#             lasts[m] = i

#     def _mask_to_delineation(self, data):
#         masks = np.argmax(data, 1)
#         delineation = []
#         v_to_del = {0:'none', 1:'p', 2:'qrs', 3:'t'}
#         for rec in masks:
#             self._merge_small(rec)
#             self._remove_small(rec)
#             rec_del = []
#             i = 0
#             rec_len = len(rec)
#             while i < rec_len:
#                 v = rec[i]
#                 if v > 0:
#                     rec_del.append({
#                         "begin": i,
#                         "end": 0,
#                         "type": v_to_del[v]
#                     })
#                     while i < rec_len and rec[i] == v:
#                         rec_del[-1]["end"] = i
#                         i += 1
#                     t = rec_del[-1]
#                     if t["begin"] < self._border or t["end"] > rec_len - self._border:
#                         rec_del.pop()
#                 i += 1
#             d_res = []
#             for c, n in zip(rec_del[:-1], rec_del[1:]):
#                 d_res.append(c)
#                 d_res.append({
#                     "begin": c["end"],
#                     "end": n["begin"],
#                     "type": "none"
#                 })
#             if rec_del:
#                 begin = {
#                     "begin": 0,
#                     "end": rec_del[0]["begin"],
#                     "type": "none"
#                 }
#                 end = {
#                     "begin": rec_del[-1]["end"],
#                     "end": rec_len,
#                     "type": "none"
#                 }
#                 d_res = [begin] + d_res + [rec_del[-1], end]
#             else:
#                 d_res.append({
#                     "begin": 0,
#                     "end": rec_len,
#                     "type": "none"
#                 })
#             delineation.append(d_res)
#         return delineation

#     def __call__(self, signal):
#         signal = torch.FloatTensor(np.expand_dims(signal, axis=1))
#         masks = self._model(signal).data.numpy()
#         return self._mask_to_delineation(masks)

# def main():
#     database_info = pd.read_csv(path_to_data+"Diagnostics.csv",delimiter=';')
#     delineation = Delineation()
#     i = 0
#     for f in database_info['FileName']:
#         signal = pd.read_csv("ECGDataDenoised/"+f+'.csv',delimiter=',', header=None)
#         data = np.array(signal) / 1000
#         data_transp = np.transpose(data)
#         result = delineation(data_transp)
#         with open(path_to_data + f + ".json", "w") as write_file:
#             json.dump(result, write_file)
#         print(i, " record passed")
#         i+=1

# if __name__ == "__main__":
#     main()




Теперь необходимо выделить нужные нам характеристики.

In [ ]:
ch_names = ['VentricularRate', 'AtrialRate', 'QRSDuration', 'QTInterval', 'QTCorrected', 'QRSCount', 'QOnset', 'QOffset', 'TOffset']
attributes = ["Gender", "PatientAge"]
for ch_names in ch_names:
    attributes.append('New_' + ch_names)
macro_data = pd.DataFrame(columns=attributes)
macro_data

Добавим из нашего датасета, с которым работали в начале, в наш новый датасет возраст и пол пациента.

In [ ]:
data = pd.read_csv("data/Diagnostics.csv",delimiter=';')
macro_data["Gender"]=data["Gender"]
macro_data["PatientAge"]=data["PatientAge"]
macro_data

Теперь выделим необходимы для работы признаки:

*Attributes* | *Type* | *Value Range* | *Description* |
--- | --- | --- | --- |
VentricularRate | Numeric | 0-999 | Ventricular rate in BPM |
AtrialRate | Numeric | 0-999 | Atrial rate in BPM |
QRSDuration | Numeric | 0-999 | QRS duration in BPM |
QTInterval | Numeric | 0-999 | QT interval in msec |
QTTCorrected | Numeric | 0-999 | Corrected QT interval in msec |
QRSCount | Numeric | 0-254 | QRS count |
QOnset | Numeric | 16 Bit Unsigned | Q onset |
QOffset | Numeric | 17 Bit Unsigned | Q offset |
TOffset | Numeric | 18 Bit Unsigned | T offset |

In [ ]:
sample_rate = 500
path_to_model = "Segmentation/"
path_to_data = "data/"

In [ ]:
import json


from ipywidgets import IntProgress
from IPython.display import display
from statistics import mean
from math import sqrt
from random import randrange

for i, record in enumerate(data.iloc):
    # итерируемся по 1 объекту
    record_data = json.load(open(path_to_data + record.FileName + ".json", 'r'))
    VentricularRate = []
    AtrialRate = []
    QRSDuration = []  # duration qrs in ms
    QTInterval = []  # numeric Interval between start of QRS and end of T wave, ms
    QTTCorrected = []  # numeric Corrected QT-interval according to Bazzet's formula, ms
    QRSCount = []
    QOnset = []
    QOffset = []
    TOffset = []
    # то есть мы будем итерироваться только по 2 отведению ?
    #for ch_num, chanel in enumerate(record_data):
        # итерируемся по 12 отведениям
        # списки длин соответствующих волн, комплексов, интервалов
    chanel = record_data[1]  # указываем второе отведение
    count_q = 0
    count_t = 0
    count_p = 0
    R_wave = [0]
    RR_interval = []
    for wave_num, wave in enumerate(chanel):
        # итерируемся по сигналу 1-го отведения
        if wave['type'] == 'p':
            count_p += 1
        if wave['type'] == 'qrs':
            R_wave.append(mean([wave['begin'], wave['end']]))
            RR_interval.append(R_wave[-1] - R_wave[-2])
            count_q += 1
            if count_q == 1:
                QOnset.append(wave['begin'])
                QOffset.append(wave['end'])
            # Собираем длины qrs комплексов в список
            QRSDuration.append(wave['end'] - wave['begin'])
            if wave_num < len(chanel) - 2:
                if (chanel[wave_num + 1]['type'] == 't'):
                    QTInterval.append(chanel[wave_num + 1]['end'] - wave['begin'])
                if (chanel[wave_num + 2]['type'] == 't'):
                    QTInterval.append(chanel[wave_num + 2]['end'] - wave['begin'])
        if wave['type'] == 't':
            count_t += 1
            if count_t == 1:
                TOffset.append(wave['end'])
    if len(QTInterval) == 0:
        QTInterval.append(0)
        RR_interval.append(1)
        QTTCorrected.append(0)
    else:
        for j in range(0, len(QTInterval)):
            if RR_interval[j] == 0:
                RR_interval[j] = mean(RR_interval)
            QTTCorrected.append(QTInterval[j] / sqrt(RR_interval[j]))
    if count_t == 0:
        TOffset.append(0) #есть сигнал плохо сегментируемый MUSE_20180114_075129_97000
    if count_q == 0:
        QOnset.append(0) #MUSE_20180712_151203_59000
        QOffset.append(0) #MUSE_20180712_151203_59000
        QRSDuration.append(0) #MUSE_20180712_151203_59000
    count_q += 2 # костыль тк сеть режет конец и начало
    QRSCount.append(count_q)
    if count_p < 5:
        AtrialRate.append(randrange(150, 400))
    else:
        AtrialRate.append(int(count_p * 6))
    #VentricularRate.append(int((len(R_wave)-1) * 6))
    VentricularRate.append(int(count_q * 6))
    macro_data.loc[macro_data.index[i], 'New_QRSCount'] = QRSCount
    macro_data.loc[macro_data.index[i], 'New_QOnset'] = QOnset
    macro_data.loc[macro_data.index[i], 'New_QOffset'] = QOffset
    macro_data.loc[macro_data.index[i], 'New_TOffset'] = TOffset
    macro_data.loc[macro_data.index[i], 'New_QTInterval'] = int(mean(QTInterval) / sample_rate * 1000)
    macro_data.loc[macro_data.index[i], 'New_QRSDuration'] = int(mean(QRSDuration) / sample_rate * 1000)
    macro_data.loc[macro_data.index[i], 'New_QTCorrected'] = int(mean(QTTCorrected) / sample_rate * 1000)
    macro_data.loc[macro_data.index[i], 'New_VentricularRate'] = VentricularRate
    macro_data.loc[macro_data.index[i], 'New_AtrialRate'] = AtrialRate
macro_data

Теперь добавим диагнозы из нашего изначального датасета.

In [ ]:
macro_data["Rhythm"] = data["Rhythm"]
macro_data.drop(macro_data.index[10606:10646],inplace=True)
macro_data

Теперь, собрав все нужные нам характеристики у нашего нового датасета, можем проверить метрики точности предсказания диагноза.

Сделаем как в прошлый раз объединение в супер классы.

In [ ]:
for i in range(0,len(macro_data["Rhythm"])):
    if macro_data['Rhythm'][i]=='AFIB' or macro_data['Rhythm'][i]=='AF':
        macro_data['Rhythm'][i]='AFIB'
    if macro_data['Rhythm'][i]=='SVT' or macro_data['Rhythm'][i]=='AT' or macro_data['Rhythm'][i]=='AVNRT' or macro_data['Rhythm'][i]=='AVRT'  or macro_data['Rhythm'][i]=='ST' or macro_data['Rhythm'][i]=='SAAWR':
        macro_data['Rhythm'][i]='GSVT'
    if macro_data['Rhythm'][i]=='SR' or macro_data['Rhythm'][i]=='SA':
        macro_data['Rhythm'][i]='SR'

In [ ]:
corr_mat = macro_data.corr()
corr_mat

In [ ]:
corr_mat.where(np.triu(corr_mat > 0.75, k=1)).stack().sort_values(ascending=False)

In [ ]:
le = preprocessing.LabelEncoder()

le.fit(macro_data['Gender'])
macro_data['Gender']=le.transform(macro_data['Gender'])
le.fit(macro_data['Rhythm'])
macro_data['Rhythm']=le.transform(macro_data['Rhythm'])
macro_data=pd.get_dummies(macro_data, columns=["Gender"])

In [ ]:
y=macro_data['Rhythm']
macro_data.pop('Rhythm')
X=macro_data

In [ ]:
X_scaled=scale(X)
X_scaled

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25,stratify=y, random_state=42)
#мое

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(12,)),
    keras.layers.Dense(600, activation=tf.nn.tanh),
    keras.layers.Dense(400, activation=tf.nn.tanh),
    keras.layers.Dense(100, activation=tf.nn.tanh),
    keras.layers.Dense(20, activation=tf.nn.tanh),
    keras.layers.Dense(4, activation=tf.nn.softmax)
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=12)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

# Вывод:

В первой части работы мы провели исследование над уже готовыми размеченными данными, которые нам предоставили Chapman University и Shaoxing People's Hospital. На этих данных точность предсказания достигла максимума (91% точности) при использовании неглубокой нейронной сети с 4-мя нелинейными слоями. Во второй части работы мы разметили вручную голый сигнал с добавлением двух признаков из размеченных данных (пол и возраст). По результатам, полученным после нашей собственной разметки данных точность на тестовой выборке достигла примерно 89%. На основании этого можно сделать вывод о том, что способ сегментации электрокардиограммы практически не влияет на точность задачи классификации заболеваний сердечно-сосудистой системы.